In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import collections
pd.set_option('display.max_columns', 500)

import pandas as pd
import tensorflow as tf
import seaborn as sns
import transformers

import nltk
import re


from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from transformers import TFXLNetModel, XLNetTokenizer

plt.style.use('seaborn')

In [2]:
df_train = pd.read_pickle('../data/train/raw.pickle')
df_train.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[{'version': 'v1', 'created': 'Wed, 10 Feb 199...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[{'version': 'v1', 'created': 'Thu, 27 Mar 201...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[{'version': 'v1', 'created': 'Thu, 22 May 201...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[{'version': 'v1', 'created': 'Tue, 3 Jul 2018...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[{'version': 'v1', 'created': 'Wed, 15 May 201...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0,NaN


In [3]:
df_train.shape

(851524, 16)

In [4]:
df_train = df_train[df_train['cites'].isnull()][['id', 'title', 'abstract', 'doi_cites']]
df_train['text'] = df_train['title'] + df_train['abstract']
df_train = df_train[['id', 'text', 'doi_cites']]
df_train.head()

,id,text,doi_cites
0,hep-ph/9902295,Heavy-Flavour Production at HERA We review th...,1
4,1905.05921,A Modified Mixed Domain Method for Modeling Ac...,0
5,1812.07485,Gaussian asymptotic limits for the $\alpha$-tr...,0
9,2005.02493,A Framework for Designing and Evaluating Solar...,0
14,astro-ph/0503092,On the deficit of calculated muon flux at sea ...,0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_train['text'].values, df_train['doi_cites'].values, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
X_train.shape, X_val.shape, X_test.shape

((501843,), (167282,), (167282,))

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer.tokenize('hello world'))

['hello', 'world']


In [7]:
def create_xlnet(mname):
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    # Define token ids as inputs
    word_inputs = tf.keras.Input(shape=(120,), name='word_inputs', dtype='int32')

    # Call XLNet model
    xlnet = TFXLNetModel.from_pretrained(mname)
    xlnet_encodings = xlnet(word_inputs)[0]

    # CLASSIFICATION HEAD 
    # Collect last step from last hidden state (CLS)
    doc_encoding = tf.squeeze(xlnet_encodings[:, -1:, :], axis=1)
    # Apply dropout for regularization
    doc_encoding = tf.keras.layers.Dropout(.1)(doc_encoding)
    # Final output 
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)

    # Compile model
    model = tf.keras.Model(inputs=[word_inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss='mse', metrics=['mes', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model
#xlnet = create_xlnet(xlnet_model)
#xlnet.summary()

In [9]:
def get_inputs(texts, tokenizer):
    """ Gets tensors from text using the tokenizer provided"""
    max_length = 172
    shape = (len(texts), max_length)
    input_ids = np.zeros(shape, dtype="int32")
    attention_mask = np.zeros(shape, dtype="int32")
    token_type_ids = np.zeros(shape, dtype="int32")
    for i, text in enumerate(texts):
        if i % 100 == 0:
            print(i)
        encoded_dict = tokenizer.encode_plus(text, max_length=max_length, pad_to_max_length=True)
        input_ids[i] = encoded_dict["input_ids"]
        attention_mask[i] = encoded_dict["attention_mask"]
        token_type_ids[i] = encoded_dict["token_type_ids"]
    return [input_ids, attention_mask, token_type_ids]

def warmup(epoch, lr):
    """Used for increasing the learning rate slowly, this tends to achieve better convergence.
    However, as we are finetuning for few epoch it's not crucial.
    """
    return max(lr +1e-6, 2e-5)

def plot_metrics(pred, true_labels):
    """Plots a ROC curve with the accuracy and the AUC"""
    acc = accuracy_score(true_labels, np.array(pred.flatten() >= .5, dtype='int'))
    fpr, tpr, thresholds = roc_curve(true_labels, pred)
    auc = roc_auc_score(true_labels, pred)

    fig, ax = plt.subplots(1, figsize=(8,8))
    ax.plot(fpr, tpr, color='red')
    ax.plot([0,1], [0,1], color='black', linestyle='--')
    ax.set_title(f"AUC: {auc}\nACC: {acc}");
    return fig
#inp_tok, ids, segments = get_inputs(X_train, xlnet_tokenizer) 

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

def build_model(transformer, num_cls=1, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(num_cls, activation='softmax')(cls_token)

    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=2e-4), loss='categorical_crossentropy', metrics=['accuracy']) # lr = 5e-5 * 4

    return model

In [17]:
def encode_texts(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        list(texts), 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    return np.array(enc_di['input_ids'], dtype = 'int32')

x_train = encode_texts(X_train, tokenizer)
x_valid = encode_texts(X_valid, tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


NameError: name 'X_valid' is not defined

In [19]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|            AutoTokenizer|      1064|
|                       In|       264|
|                      Out|       360|
|                     Path|       896|
|             TFXLNetModel|      1064|
|           XLNetTokenizer|      1064|
|                   X_test|   1338352|
|                  X_train|   4014840|
|                    X_val|   1338352|
|           accuracy_score|       136|
|              collections|        72|
|             create_xlnet|       136|
|                 df_train|1008368901|
|             encode_texts|       136|
|                     exit|        48|
|               get_inputs|       136|
|              get_ipython|        64|
|                     nltk|        72|
|                       np|        72|
|                       os|        72|
|                       pd|        72|
|             plot_metrics|       136|
|                      plt|        72|
|                     qui

In [20]:
1027774576 / 1000000000

1.027774576

In [14]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|            AutoTokenizer|      1064|
|                       In|       192|
|                      Out|       360|
|                     Path|       896|
|             TFXLNetModel|      1064|
|           XLNetTokenizer|      1064|
|                   X_test|   1338352|
|                  X_train|   4014840|
|                    X_val|   1338352|
|           accuracy_score|       136|
|              collections|        72|
|             create_xlnet|       136|
|                 df_train|1008368901|
|             encode_texts|       136|
|                     exit|        48|
|               get_inputs|       136|
|              get_ipython|        64|
|                     nltk|        72|
|                       np|        72|
|                       os|        72|
|                       pd|        72|
|             plot_metrics|       136|
|                      plt|        72|
|                     qui

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=4, min_delta=0.02, restore_best_weights=True),
    tf.keras.callbacks.LearningRateScheduler(warmup, verbose=0),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mse', factor=1e-6, patience=2, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-6)
]

In [ ]:
#hist = xlnet.fit(x=inp_tok, y=y_train, epochs=15, batch_size=16, validation_split=.15, callbacks=callbacks)